# Pair Programming ETL Carga I

Igual que en el ejercicio de pair programming de ETL Transformación II, tendréis que crear una clase que nos permita cargar los datos en SQL I.
Happy coding 🦈

In [8]:
import pandas as pd
import mysql.connector

In [9]:
class Cargar:
    
    def __init__(self, nombre_bbdd, contraseña):
        # nuestra clase va a recibir dos parámetros que son fijos a lo largo de toda la BBDD, el nombre de la BBDD y la contraseña con el servidor. 
        self.nombre_bbdd = nombre_bbdd
        self.contraseña = contraseña

    # método para crear la BBDD 
    def crear_bbdd(self):

        mydb = mysql.connector.connect(host="localhost",
                                       user="root",
                                       password=f'{self.contraseña}') 
        mycursor = mydb.cursor()
        print("Conexión realizada con éxito")

        try:
            mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {self.nombre_bbdd};")
            
        except:
            print("La BBDD ya existe")
    
    # método para crear tablas  e insertar datos en ellas   
    def crear_insertar_tabla(self, query):
        
        mydb = mysql.connector.connect(host="localhost",
                                       user="root",
                                       password=f'{self.contraseña}', 
                                       database=f"{self.nombre_bbdd}") 
        mycursor = mydb.cursor()
        
        try:
            mycursor.execute(query)
            mydb.commit()
          
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)

    

Llamamos a la clase

In [10]:
db_sql = Cargar("tiburones2", "AlumnaAdalab")

Aplicamos el método crear_bbdd para crear la base de datos

In [11]:
mydb = db_sql.crear_bbdd()

Conexión realizada con éxito


Aplicamos el método para crear las tablas. Con el método crear_insertar_tabla(self, query)

In [18]:
# query crear tabla ataques
tabla_ataques = '''
CREATE TABLE IF NOT EXISTS `tiburones2`.`ataques` (
  `case_number` VARCHAR(50) NOT NULL, 
  `year` INT NOT NULL , 
  `type` VARCHAR(50) NOT NULL, 
  `country` VARCHAR(50) NOT NULL, 
  `species_` VARCHAR(50),
  `date` VARCHAR(50),
  `mes` VARCHAR(50),
  `fatal` VARCHAR(50),
  `sexo` VARCHAR(50),
  `especie_cat`VARCHAR(50),
  `age` FLOAT (6),
  
  PRIMARY KEY (`case_number`))
ENGINE = InnoDB;
'''
# El ENGINE, es el motor de almacenamiento, el encargado de almacenar y manejar la información de la tabla.

In [19]:
# query crear tabla clima

tabla_clima = '''
CREATE TABLE IF NOT EXISTS `tiburones2`.`clima`(
  `timepoint` INT NOT NULL, 
  `cloudcover` INT NOT NULL, 
  `highcloud` INT NOT NULL,
  `midcloud` INT NOT NULL, 
  `lowcloud` INT NOT NULL, 
  `rh_profile` VARCHAR (1000),
  `wind_profile` VARCHAR (1000),
  `temp2m` INT NOT NULL, 
  `lifted_index` INT NOT NULL, 
  `rh2m` INT NOT NULL,
  `msl_pressure` INT NOT NULL, 
  `prec_type` VARCHAR (1000),
  `prec_amount` INT (50) NOT NULL, 
  `snow_depth` INT (50) NOT NULL, 
  `wind10m.direction` INT NOT NULL,
  `wind10m.speed` INT NOT NULL,
  `pais` VARCHAR (50) NOT NULL,
  `case_number` VARCHAR (50) NOT NULL,
    
  PRIMARY KEY ( `pais`),
  INDEX `fk_clima_ataques_idx` (`pais` ASC),
  CONSTRAINT `fk_clima_ataques`
    FOREIGN KEY (`case_number`)
    REFERENCES `tiburones2`.`ataques` (`case_number`))
ENGINE = InnoDB;
'''

In [20]:
db_sql.crear_insertar_tabla(tabla_ataques)

In [21]:
db_sql.crear_insertar_tabla(tabla_clima)

Inserción de datos con el método crear_insertar_tabla()

In [ ]:
for indice, fila in df.iterrows(): 
    
    query_tiempo = f"""
            INSERT INTO tiempo (timepoint, fecha) # como la primary key la hemos puesto como autoincremental no hace falta introducirla en el insert
            VALUES ( "{fila["timepoint"]}", "{fila['fecha']}");
            """
    carga.crear_insertar_tabla(query_tiempo)

In [ ]:
query_ataques = f"""
                INSERT INTO mediciones (cloudcover, lifted_index, prec_type, prec_amount, temp2m, rh2m, weather, wind_direction, wind_speed, seeing, transparency, idtiempo, idlocalidad) 
                VALUES ({fila["cloudcover"]}, {fila['lifted_index']}, "{fila['prec_type']}", {fila['prec_amount']},{fila['temp2m']},{fila['rh2m']}, "{fila['weather']}","{fila['wind10m.direction']}",{fila['wind10m.speed']}, {fila['seeing']}, {fila['transparency']}, {id_fecha}, {id_ciudad});
                """

  `timepoint` INT NOT NULL, 
  `cloudcover` INT NOT NULL, 
  `highcloud` INT NOT NULL,
  `midcloud` INT NOT NULL, 
  `lowcloud` INT NOT NULL, 
  `rh_profile` VARCHAR (1000),
  `wind_profile` VARCHAR (1000),
  `temp2m` INT NOT NULL, 
  `lifted_index` INT NOT NULL, 
  `rh2m` INT NOT NULL,
  `msl_pressure` INT NOT NULL, 
  `prec_type` VARCHAR (1000),
  `prec_amount` INT (50) NOT NULL, 
  `snow_depth` INT (50) NOT NULL, 
  `wind10m.direction` INT NOT NULL,
  `wind10m.speed` INT NOT NULL,
  `pais` VARCHAR (50) NOT NULL,
  `case_number` VARCHAR (50) NOT NULL,

In [ ]:
query_clima = f"""
                INSERT INTO clima (timepoint, cloudcover, highcloud, midcloud, lowcloud, rh_profile,wind_profile, temp2m, lifted_index, rh2m,prec_type, prec_amount, temp2m,  weather, wind_direction, wind_speed, seeing, transparency, idtiempo, idlocalidad) 
                VALUES ({fila["timepoint"]}, {fila["cloudcover"]}, {fila["highcloud"]},{fila["midcloud"]}, {fila["lowcloud"]}, {fila["rh_profile"]} , {fila["wind_profile"]}, {fila["temp2m"]},{fila['lifted_index']}, {fila['rh2m']}, "{fila['prec_type']}", {fila['prec_amount']},{fila['temp2m']}, "{fila['weather']}","{fila['wind10m.direction']}",{fila['wind10m.speed']}, {fila['seeing']}, {fila['transparency']}, {id_fecha}, {id_ciudad});
                """

In [ ]:
 prec_amount, temp2m,  weather, , seeing, transparency, idtiempo, idlocalidad) 

In [ ]:
"{fila['weather']}", {fila['seeing']}, {fila['transparency']}, {id_fecha}, {id_ciudad})

In [ ]:
query_clima = f"""
                INSERT INTO clima (timepoint, cloudcover, highcloud, midcloud, lowcloud, rh_profile, wind_profile, temp2m, lifted_index, rh2m,prec_type, prec_amount, `snow_depth`, wind_direction, wind_speed
                VALUES ({fila["timepoint"]}, {fila["cloudcover"]}, {fila["highcloud"]},{fila["midcloud"]}, {fila["lowcloud"]}, {fila["rh_profile"]} , {fila["wind_profile"]}, {fila["temp2m"]},{fila['lifted_index']}, {fila['rh2m']}, "{fila['prec_type']}", {fila['prec_amount']}, "{fila['wind10m.direction']}",{fila['wind10m.speed']});
                """

In [ ]:

  `wind10m.direction` INT NOT NULL,
  `wind10m.speed` INT NOT NULL,
  `pais` VARCHAR (50) NOT NULL,
  `case_number` VARCHAR (50) NOT NULL,

In [ ]:
  `msl_pressure` INT NOT NULL, 
  `prec_type` VARCHAR (1000),
  `prec_amount` INT (50) NOT NULL, 
  `snow_depth` INT (50) NOT NULL, 
  `wind10m.direction` INT NOT NULL,
  `wind10m.speed` INT NOT NULL,
  `pais` VARCHAR (50) NOT NULL,
  `case_number` VARCHAR (50) NOT NULL,

In [1]:
# método para comprobar si la ciudad que estamos intentando meter ya existe en la BBDD. 
    # En caso de que ya exista no será necesario que la insertemos de nuevo
def check_paises(self):

    mydb = mysql.connector.connect(user='root',
                                    password=f"{self.contraseña}",
                                    host='127.0.0.1',
                                    database=f"{self.nombre_bbdd}")
    mycursor = mydb.cursor()

    # query para extraer los valores únicos de ciudades de la tabla de localidades 
    query_existe_ciudad = f"""
            SELECT DISTINCT country FROM ataques
            """
    mycursor.execute(query_existe_ciudad)
    ciudades = mycursor.fetchall()
    return ciudades

Otras funciones

In [2]:
def sacar_id_ciudad(self, ciudad):
        
    mydb = mysql.connector.connect(user='root',
                                    password= f'{self.contraseña}',
                                    host='127.0.0.1', 
                                    database=f"{self.nombre_bbdd}")
    mycursor = mydb.cursor()
    
    try:
        query_sacar_id = f"SELECT idlocalidad FROM localidades WHERE ciudad = '{ciudad}'"
        mycursor.execute(query_sacar_id)
        id_ = mycursor.fetchall()[0][0]
        return id_
    
    except: 
        return "Sorry, no tenemos esa ciudad en la BBDD y por lo tanto no te podemos dar su id. "

In [3]:
def sacar_id_fecha(self, timepoint, fecha):
    mydb = mysql.connector.connect(user='root', password=f'{self.contraseña}',
                                        host='127.0.0.1', database=f"{self.nombre_bbdd}")
    mycursor = mydb.cursor()

    try:
        query_sacar_id = f"SELECT idtiempo FROM tiempo WHERE timepoint = {timepoint} and fecha = '{fecha}'"
        mycursor.execute(query_sacar_id)
        id_ = mycursor.fetchall()[0][0]
        return id_
    
    except: 
            return "Sorry, no tenemos esa fecha en la BBDD y por lo tanto no te podemos dar su id. "